In [1]:
import pandas as pd
import os
import numpy as np

import Utils as ut
import CortesAlignmentFile as ca
import mySampler as ms
 
from sklearn.model_selection import train_test_split

/Users/danilo/.conda/envs/bcb/lib/python3.7/site-packages/sklearn/utils/__init__.py:4: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import Sequence


In [2]:
d_clinical = pd.read_csv(os.path.join('data', 'dataset_clinical_cleaned.csv'))
d_genetic = pd.read_csv(os.path.join('data', 'dataset_genetic_cleaned_noOHE.csv'))
d_vampire = pd.read_csv(os.path.join('data', 'dataset_vampire_cleaned.csv'))
outputs = pd.read_csv(os.path.join('data', 'outputs_cleaned.csv'))

## Heart Attack

In [3]:
y_class = outputs["cvd_fail"].values
meaningful_idxs = np.where(y_class==1)
y = outputs["cvd_time_age"].values[meaningful_idxs]
C = d_clinical.values[meaningful_idxs]
G = d_genetic.values[meaningful_idxs]
V = d_vampire.values[meaningful_idxs]

In [4]:
# COMPUTATIONAL COMPLEXITY: Reduce #samples
#tr_idx, ts_idx = next(StratifiedShuffleSplit(n_splits=1, test_size=0.75).split(C, y))
C_, C_test, G_, G_test, V_, V_test, y_, y_test = train_test_split(C, G, V, y, test_size=0.25)

In [5]:
ds_list = [C_, G_, V_]
ds_test = [C_test, G_test, V_test]
ds_names = ['clinic', 'genetic', 'vampire']

## Basic approach

In [6]:
kernel_names = ['linear', 'polynomial', 'gaussian']
kernel_type = [{'linear':[0.5], 'polynomial':[2, 3, 7], 'gaussian':[0.1, 0.5, 0.7]},
               {'linear':[0.2], 'polynomial':[4, 5, 8], 'gaussian':[0.7, 1]}]
estimator = ca.centeredKernelAlignment
sampler = ms.mySampler(n_splits=3, test_size=.25, Ptype="regression")

In [7]:
result1 = sampler.sample(kernel_type, estimator, ds_list, y_, valid_fold = 3, verbose=True)


1 split out of 3 ...
	Working on config 1 of 2: {'linear': [0.5], 'polynomial': [2, 3, 7], 'gaussian': [0.1, 0.5, 0.7]}
Fold no. 1
		Perfomances computed for 200
		Perfomances computed for 400
		Perfomances computed for 600
Fold no. 2
		Perfomances computed for 200
		Perfomances computed for 400
		Perfomances computed for 600
Fold no. 3
		Perfomances computed for 200
		Perfomances computed for 400
		Perfomances computed for 600
Validation complete, config selected:[[0.5, 7, 0.7], [0.5, 2, 0.1], [0.5, 3, 0.1]]
	Result of 1:
CA: 0.006068261932881248
Average error: 7.077805240725186
Error variance: 24.4968317344937
[linear:0.5, polynomial:7, gaussian:0.7, ]
[linear:0.5, polynomial:2, gaussian:0.1, ]
[linear:0.5, polynomial:3, gaussian:0.1, ]

eta vector: [ 2.70647630e-05 -1.54175192e-26 -7.30222214e-01 -6.36514498e-06
 -1.90849414e-04  6.25446030e-02 -1.75784823e-12  4.13411021e-26
  6.80340836e-01]


	Completed in 3.6666666666666665 minutes
	Working on config 2 of 2: {'linear': [0.2], '

In [8]:
result1.global_best_

[[{'CA': 0.006068261932881248,
   'meanErr': 7.077805240725186,
   'varErr': 24.4968317344937,
   'config': <KernelFile.kernelWrapper at 0x10c5ad470>,
   'eta': array([ 2.70647630e-05, -1.54175192e-26, -7.30222214e-01, -6.36514498e-06,
          -1.90849414e-04,  6.25446030e-02, -1.75784823e-12,  4.13411021e-26,
           6.80340836e-01])},
  {'CA': 0.005783943977616463,
   'meanErr': 7.256503761614171,
   'varErr': 32.255454461079516,
   'config': <KernelFile.kernelWrapper at 0x1a1ccce208>,
   'eta': array([ 2.81396186e-06, -3.56111252e-31,  6.23134912e-01, -3.14556159e-07,
          -6.95783940e-06, -7.68235221e-01, -2.75908439e-13,  2.77401178e-42,
           1.46688537e-01])}],
 [{'CA': 0.008169641633632348,
   'meanErr': 7.484993741868421,
   'varErr': 22.59249598681801,
   'config': <KernelFile.kernelWrapper at 0x1a1cd42e80>,
   'eta': array([ 6.75914231e-07, -1.32935955e-09,  3.65123301e-04,  6.22212825e-09,
          -2.95874462e-08,  7.06942122e-01,  3.60451513e-14, -1.142001

In [9]:
w_dict, w_list = result1.votingOverCA(ds_names, kernel_names)
ut.testConfigurations(estimator, y_, y_test, w_list, ds_list, ds_test, kernel_names)

ValueError: Classification metrics can't handle a mix of continuous and binary targets

In [ ]:
w_list

In [ ]:
w_dict

In [10]:
result1.performancesFeatures()

statistics of configuration 1
{'config': {'linear': [[0.5, 0.5, 0.5], [0.5, 0.5, 0.5], [0.5, 0.5, 0.5]], 'polynomial': [[7, 2, 3], [2, 7, 2], [7, 3, 7]], 'gaussian': [[0.7, 0.1, 0.1], [0.1, 0.7, 0.7], [0.1, 0.7, 0.5]]}, 'CA': (0.006642437108682686, 1.1900985779570726e-06), 'meanErr': (7.417687565854261, 0.06478262444370342), 'varErr': (26.747669931947836, 21.12291155877918), 'eta': (array([ 9.27599838e-06, -4.43119851e-10, -2.43230844e-01, -2.11219809e-06,
       -6.40022101e-05,  4.92176150e-01, -5.49078486e-13, -3.80666974e-20,
       -2.44700762e-01]), array([1.58277814e-10, 3.92710405e-19, 1.18580304e-01, 9.04382187e-12,
       8.04530750e-09, 9.22916346e-02, 7.30809480e-25, 2.89815005e-39,
       4.27850981e-01]))}
statistics of configuration 2
{'config': {'linear': [[0.2, 0.2, 0.2], [0.2, 0.2, 0.2], [0.2, 0.2, 0.2]], 'polynomial': [[8, 4, 5], [8, 4, 5], [8, 4, 8]], 'gaussian': [[0.7, 1, 1], [0.7, 0.7, 0.7], [1, 1, 1]]}, 'CA': (0.005619522206681812, 2.3721308836389062e-08), 'meanE

## Kernel normalization

In [ ]:
kernel_names = ['linear', 'polynomial', 'gaussian']
kernel_type = [{'linear':[0.5], 'polynomial':[2, 3], 'gaussian':[0.1, 0.3, 0.6]}#,
               #{'linear':[0.2], 'polynomial':[4, 5, 8], 'gaussian':[0.7, 1]}
              ]
estimator = ca.centeredKernelAlignment
sampler = ms.mySampler(n_splits=3, test_size=.25, normalize_kernels = True)

In [ ]:
result3 = sampler.sample(kernel_type, estimator, ds_list, y_, valid_fold = 3, verbose=True)

In [ ]:
w_dict, w_list = result3.votingOverCA(ds_names, kernel_names)
ut.testConfigurations(estimator, y_, y_test, w_list, ds_list, ds_test, kernel_names)

In [ ]:
result3.performancesFeatures()

## Normalized data

In [ ]:
kernel_names = ['linear', 'polynomial', 'gaussian']
kernel_type = [{'linear':[0.5], 'polynomial':[2, 3, 7], 'gaussian':[0.1, 0.5, 0.7]},
               {'linear':[0.2], 'polynomial':[4, 5, 8], 'gaussian':[0.7, 1]}]
estimator = ca.centeredKernelAlignment
sampler = ms.mySampler(n_splits=3, test_size=.25, normalizing = True)

In [ ]:
result4 = sampler.sample(kernel_type, estimator, ds_list, y_, valid_fold = 3, verbose=True, exclusion_list = [[0, 3, 8, 13, 14], list(range(G.shape[1]-3)), []])

In [ ]:
w_dict, w_list = result4.votingOverCA(ds_names, kernel_names)
ut.testConfigurations(estimator, y_, y_test, w_list, ds_list, ds_test, kernel_names)

In [ ]:
result4.performancesFeatures()

## Origin Data Centering

In [ ]:
kernel_names = ['linear', 'polynomial', 'gaussian']
kernel_type = [{'linear':[0.5], 'polynomial':[2, 3, 7], 'gaussian':[0.1, 0.5, 0.7]},
               {'linear':[0.2], 'polynomial':[4, 5, 8], 'gaussian':[0.7, 1]}]
estimator = ca.centeredKernelAlignment
sampler = ms.mySampler(n_splits=3, test_size=.25, centering = True)

In [ ]:
result5 = sampler.sample(kernel_type, estimator, ds_list, y_, valid_fold = 3, verbose=True, exclusion_list = [[5,6,7,9,10,13,15,16,17,18,19], list(range(G.shape[1]-3)), []])

In [ ]:
w_dict, w_list = result5.votingOverCA(ds_names, kernel_names)
ut.testConfigurations(estimator, y_, y_test, w_list, ds_list, ds_test, kernel_names)

In [ ]:
result5.performancesFeatures()

## Origin Data  Centering and Normalization

In [ ]:
kernel_names = ['linear', 'polynomial', 'gaussian']
kernel_type = [{'linear':[0.5], 'polynomial':[2, 3, 7], 'gaussian':[0.1, 0.5, 0.7]},
               {'linear':[0.2], 'polynomial':[4, 5, 8], 'gaussian':[0.7, 1]}]
estimator = ca.centeredKernelAlignment
sampler = ms.mySampler(n_splits=3, test_size=.25, centering = True, normalizing = True)

In [ ]:
result6 = sampler.sample(kernel_type, estimator, ds_list, y_, valid_fold = 3, verbose=True, exclusion_list = [[5,6,7,9,10,13,15,16,17,18,19], list(range(G.shape[1]-3)), []])

In [ ]:
w_dict, w_list = result6.votingOverCA(ds_names, kernel_names)
ut.testConfigurations(estimator, y_, y_test, w_list, ds_list, ds_test, kernel_names)

In [ ]:
result6.performancesFeatures()

## Normalization, Kernel Normalization

In [ ]:
kernel_names = ['linear', 'polynomial', 'gaussian']
kernel_type = [{'linear':[0.5], 'polynomial':[2, 3,4], 'gaussian':[0.1, 0.3, 0.6]},
               {'linear':[0.2], 'polynomial':[4, 5, 8], 'gaussian':[0.7, 1]}
              ]
estimator = ca.centeredKernelAlignment
sampler = ms.mySampler(n_splits=3, test_size=.25, normalizing = True, normalize_kernels = True)

In [ ]:
result10 = sampler.sample(kernel_type, estimator, ds_list, y_, valid_fold = 3, verbose=True, exclusion_list = [[5,6,7,9,10,13,15,16,17,18,19], list(range(G.shape[1]-3)), []])

In [ ]:
w_dict, w_list = result10.votingOverCA(ds_names, kernel_names)
ut.testConfigurations(estimator, y_, y_test, w_list, ds_list, ds_test, kernel_names)

In [ ]:
result10.performancesFeatures()

## Centering, Kernel Normalization

In [ ]:
kernel_names = ['linear', 'polynomial', 'gaussian']
kernel_type = [{'linear':[0.5], 'polynomial':[2, 3,4], 'gaussian':[0.1, 0.3, 0.6]},
               {'linear':[0.2], 'polynomial':[4, 5, 8], 'gaussian':[0.7, 1]}
              ]
estimator = ca.centeredKernelAlignment
sampler = ms.mySampler(n_splits=3, test_size=.25, centering = True, normalize_kernels = True)

In [ ]:
result11 = sampler.sample(kernel_type, estimator, ds_list, y_, valid_fold = 3, verbose=True, exclusion_list = [[5,6,7,9,10,13,15,16,17,18,19], list(range(G.shape[1]-3)), []])

In [ ]:
w_dict, w_list = result11.votingOverCA(ds_names, kernel_names)
ut.testConfigurations(estimator, y_, y_test, w_list, ds_list, ds_test, kernel_names)

In [ ]:
result11.performancesFeatures()

## Centering, Normalization, Kernel Normalization

In [ ]:
kernel_names = ['linear', 'polynomial', 'gaussian']
kernel_type = [{'linear':[0.5], 'polynomial':[2, 3,4], 'gaussian':[0.1, 0.3, 0.6]},
               {'linear':[0.2], 'polynomial':[4, 5, 8], 'gaussian':[0.7, 1]}
              ]
estimator = ca.centeredKernelAlignment
sampler = ms.mySampler(n_splits=3, test_size=.25, centering = True, normalizing = True, normalize_kernels = True)

In [ ]:
result7 = sampler.sample(kernel_type, estimator, ds_list, y_, valid_fold = 3, verbose=True, exclusion_list = [[5,6,7,9,10,13,15,16,17,18,19], list(range(G.shape[1]-3)), []])

In [ ]:
w_dict, w_list = result7.votingOverCA(ds_names, kernel_names)
ut.testConfigurations(estimator, y_, y_test, w_list, ds_list, ds_test, kernel_names)

In [ ]:
result7.performancesFeatures()

## L2 Penalty, Centering, K-Normalization

In [ ]:
kernel_names = ['linear', 'polynomial', 'gaussian']
kernel_type = [{'linear':[0.5], 'polynomial':[2, 3,4], 'gaussian':[0.1, 0.3, 0.6]},
               {'linear':[0.2], 'polynomial':[4, 5, 8], 'gaussian':[0.7, 1]}
              ]
estimator = ca.centeredKernelAlignment
sampler = ms.mySampler(n_splits=3, test_size=.25, lamb = 0.5, centering = True, normalize_kernels = False)

In [ ]:
result9 = sampler.sample(kernel_type, estimator, ds_list, y_, valid_fold = 3, verbose=True, exclusion_list = [[5,6,7,9,10,13,15,16,17,18,19], list(range(G.shape[1]-3)), []])

In [ ]:
w_dict, w_list = result9.votingOverCA(ds_names, kernel_names)
ut.testConfigurations(estimator, y_, y_test, w_list, ds_list, ds_test, kernel_names)

In [ ]:
result9.performancesFeatures()

## Sparsity in eta, Centering, K-Normalization

In [ ]:
y = outputs["cvd_fail"].values
C = d_clinical.values
G = d_genetic.values
V = d_vampire.values

In [ ]:
tr_idx, ts_idx = next(StratifiedShuffleSplit(n_splits=1, test_size=0.95).split(C, y))

In [ ]:
y_ = y[tr_idx]
y_test = y[ts_idx]
C_ = C[tr_idx]
C_test = C[ts_idx]
G_ = G[tr_idx]
G_test = G[ts_idx]
V_ = V[tr_idx]
V_test = V[ts_idx]

In [ ]:
ds_list = [C_, G_, V_]
ds_test = [C_test, G_test, V_test]
ds_names = ['clinic', 'genetic', 'vampire']

In [ ]:
kernel_names = ['linear', 'polynomial', 'gaussian']
kernel_type = [{'linear':[0.5], 'polynomial':[2, 3], 'gaussian':[0.1, 0.3]}#,
               #{'linear':[0.2], 'polynomial':[4, 5, 8], 'gaussian':[0.7, 1]}
              ]
estimator = ca.centeredKernelAlignment
sampler = ms.mySampler(n_splits=3, test_size=.25, sparsity = 0.3, centering = True, normalize_kernels = True)

In [ ]:
result8 = sampler.sample(kernel_type, estimator, ds_list, y, valid_fold = 3, verbose=True, exclusion_list = [[5,6,7,9,10,13,15,16,17,18,19], list(range(G.shape[1]-3)), []])

In [ ]:
w_dict, w_list = result8.votingOverCA(ds_names, kernel_names)
ut.testConfigurations(estimator, y_, y_test, w_list, ds_list, ds_test, kernel_names)

In [ ]:
result8.performancesFeatures()